In [2]:
# General purpose libraries.
import pandas as pd
import os
import numpy as np

# Geospatial libraries.
import geopandas as gpd
import rasterio
from rasterio.plot import show
import folium
import shapely

# Voronoi and Zonal statistics libraries
from rasterstats import zonal_stats

# Visualization libraries.
from IPython.display import display
from matplotlib import pyplot as plt
import matplotlib.colors as mcolors

In [9]:
data_path = os.path.join(os.getcwd(),"data")

Census_Blocks = gpd.read_file(
    os.path.join(data_path, "Census_Block_Stats_with_Precinct_WakeCounty.geojson")
)
# 2-Voting precincts for single county
Voting_Precincts = gpd.read_file(
    os.path.join(data_path, "Voting_Precincts_WakeCounty.geojson")
)    

# 3- Polling places
Polling_Places = gpd.read_file(os.path.join(data_path, "Polling_Places_WakeCounty.geojson"))

In [10]:
display(Census_Blocks)

,STATEFP20,COUNTYFP20,TRACTCE20,BLOCKCE20,GEOID20,NAME20,MTFCC20,UR20,UACE20,UATYPE20,...,g20221108_pct_voted_all,g20201103_voted_all,g20201103_reg_all,g20201103_pct_voted_all,pp20200303_voted_all,pp20200303_reg_all,pp20200303_pct_voted_all,prec_id,enr_desc,geometry
0,37,183,052814,2019,371830528142019,Block 2019,G5040,None,None,None,...,1.0000,2,2,1.0000,2,2,1.0000,16-06,PRECINCT 16-06,"MULTIPOLYGON (((-78.60388 35.69985, -78.60386 ..."
1,37,183,054115,3006,371830541153006,Block 3006,G5040,None,None,None,...,NaN,0,0,NaN,0,0,NaN,17-13,PRECINCT 17-13,"MULTIPOLYGON (((-78.50644 35.79411, -78.50615 ..."
2,37,183,053517,2002,371830535172002,Block 2002,G5040,None,None,None,...,0.4303,182,223,0.8161,69,203,0.3399,04-11,PRECINCT 04-11,"MULTIPOLYGON (((-78.77280 35.78943, -78.77258 ..."
3,37,183,051700,3012,371830517003012,Block 3012,G5040,None,None,None,...,0.8438,56,60,0.9333,24,58,0.4138,01-11,PRECINCT 01-11,"MULTIPOLYGON (((-78.65968 35.82745, -78.65963 ..."
4,37,183,053524,2013,371830535242013,Block 2013,G5040,None,None,None,...,0.7576,233,254,0.9173,127,241,0.5270,04-20,PRECINCT 04-20,"MULTIPOLYGON (((-78.80363 35.78360, -78.80363 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14465,37,183,054212,2005,371830542122005,Block 2005,G5040,None,None,None,...,NaN,0,0,NaN,0,0,NaN,19-15,PRECINCT 19-15,"MULTIPOLYGON (((-78.55845 35.97001, -78.55811 ..."
14466,37,183,054112,2001,371830541122001,Block 2001,G5040,None,None,None,...,0.5735,114,129,0.8837,51,118,0.4322,17-15,PRECINCT 17-15,"MULTIPOLYGON (((-78.47286 35.83428, -78.47275 ..."
14467,37,183,053203,2042,371830532032042,Block 2042,G5040,None,None,None,...,0.6571,27,30,0.9000,14,29,0.4828,06-06,PRECINCT 06-06,"MULTIPOLYGON (((-78.82627 35.65693, -78.82608 ..."
14468,37,183,053617,2058,371830536172058,Block 2058,G5040,None,None,None,...,NaN,0,0,NaN,0,0,NaN,20-16,PRECINCT 20-16,"MULTIPOLYGON (((-78.90122 35.82367, -78.90069 ..."


In [11]:
display(Voting_Precincts)

,objectid,id,prec_id,enr_desc,county_nam,of_prec_id,county_id,st_areasha,st_perimet,geometry
0,2391,105,01-05,PRECINCT 01-05,WAKE,None,92,1.325901e+07,14500.105940,"MULTIPOLYGON (((641594.00373 227539.28146, 641..."
1,2392,110,01-09,PRECINCT 01-09,WAKE,None,92,8.161909e+06,12038.234692,"MULTIPOLYGON (((642164.74203 227423.62006, 642..."
2,2393,99,01-01,PRECINCT 01-01,WAKE,None,92,1.549578e+07,16835.331171,"MULTIPOLYGON (((640215.53293 225960.36126, 640..."
3,2394,101,01-02,PRECINCT 01-02,WAKE,None,92,2.445547e+07,24437.015415,"MULTIPOLYGON (((639157.44453 226581.77516, 639..."
4,2395,102,01-03,PRECINCT 01-03,WAKE,None,92,2.242057e+07,21790.186717,"MULTIPOLYGON (((638981.54073 227575.39086, 638..."
...,...,...,...,...,...,...,...,...,...,...
205,2616,2678,19-23,PRECINCT 19-23,WAKE,None,92,1.156502e+08,59499.621249,"MULTIPOLYGON (((654458.80063 244766.87386, 654..."
206,2617,2679,03-01,PRECINCT 03-01,WAKE,None,92,1.774079e+08,70646.208753,"MULTIPOLYGON (((619884.62033 216652.84496, 619..."
207,2618,2680,06-11,PRECINCT 06-11,WAKE,None,92,1.483951e+08,57886.925340,"MULTIPOLYGON (((623301.28703 211631.05966, 623..."
208,2619,2681,17-15,PRECINCT 17-15,WAKE,None,92,2.522078e+08,72688.452040,"MULTIPOLYGON (((655931.27883 227355.33396, 655..."


In [12]:
display(Polling_Places)

,FID,Status,Score,Match_type,Match_addr,Addr_type,Place_addr,Side,StPreDir,StPreType,...,USER_polli,USER_pol_1,USER_preci,USER_house,USER_stree,USER_addre,USER_city,USER_state,USER_zip,geometry
0,2001,M,100.0,A,"4216 Kildaire Farm Rd, Apex, North Carolina, 2...",PointAddress,"4216 Kildaire Farm Rd, Apex, North Carolina, 2...",None,None,None,...,193,TRIANGLE COMMUNITY CHURCH,PRECINCT 06-07,4216,KILDAIRE FARM RD,4216 KILDAIRE FARM RD,APEX,NC,27539,POINT (-8771656.39633 4257269.09596)
1,2002,M,100.0,A,"950 Holly Springs Rd, Holly Springs, North Car...",PointAddress,"950 Holly Springs Rd, Holly Springs, North Car...",None,None,None,...,191,HOLLY RIDGE MIDDLE SCHOOL,PRECINCT 06-05,950,HOLLY SPRINGS RD,950 HOLLY SPRINGS RD,HOLLY SPRINGS,NC,27540,POINT (-8773678.79629 4253134.79793)
2,2003,M,100.0,A,"605 Bridge St, Fuquay Varina, North Carolina, ...",PointAddress,"605 Bridge St, Fuquay Varina, North Carolina, ...",None,None,None,...,194,ST AUGUSTA MISSIONARY BAPTIST,PRECINCT 12-08,605,BRIDGE ST,605 BRIDGE ST,FUQUAY VARINA,NC,27526,POINT (-8773049.61775 4244687.20690)
3,2004,M,100.0,A,"401 Holly Springs Rd, Holly Springs, North Car...",PointAddress,"401 Holly Springs Rd, Holly Springs, North Car...",None,None,None,...,192,HOLLY SPRINGS ELEMENTARY SCHOO,PRECINCT 06-06,401,HOLLY SPRINGS RD,401 HOLLY SPRINGS RD,HOLLY SPRINGS,NC,27540,POINT (-8775133.30233 4253421.95680)
4,2005,M,100.0,A,"2255 Herbert Akins Rd, Fuquay Varina, North Ca...",PointAddress,"2255 Herbert Akins Rd, Fuquay Varina, North Ca...",None,None,None,...,195,HERBERT AKINS ROAD ELEMENTARY,PRECINCT 12-09,2255,HERBERT AKINS RD,2255 HERBERT AKINS RD,FUQUAY VARINA,NC,27526,POINT (-8772361.32996 4248439.57633)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,2658,M,100.0,A,"2100 Morrisville Pkwy, Cary, North Carolina, 2...",PointAddress,"2100 Morrisville Pkwy, Cary, North Carolina, 2...",None,None,None,...,189,CARPENTER ELEMENTARY SCHOOL,PRECINCT 05-06,2100,MORRISVILLE PKWY,2100 MORRISVILLE PKWY,CARY,NC,27519,POINT (-8779365.44505 4275112.11362)
202,2659,M,100.0,A,"8228 Hebron Church Rd, Garner, North Carolina,...",PointAddress,"8228 Hebron Church Rd, Garner, North Carolina,...",None,None,None,...,183,SOUTH GARNER HIGH SCHOOL,PRECINCT 16-09,8228,HEBRON CHURCH RD,8228 HEBRON CHURCH RD,GARNER,NC,27529,POINT (-8747256.17217 4253192.10048)
203,2660,M,100.0,A,"301 Stinson Ave, Holly Springs, North Carolina...",PointAddress,"301 Stinson Ave, Holly Springs, North Carolina...",None,None,None,...,190,HUNT COMMUNITY CENTER,PRECINCT 06-04,301,STINSON AVE,301 STINSON AVE,HOLLY SPRINGS,NC,27540,POINT (-8775287.75321 4252012.76123)
204,2661,M,100.0,A,"3808 Saint James Church Rd, Raleigh, North Car...",PointAddress,"3808 Saint James Church Rd, Raleigh, North Car...",None,None,None,...,185,ST JAMES UNITED METHODIST CHUR,PRECINCT 17-11,3808,ST JAMES CHURCH RD,3808 ST JAMES CHURCH RD,RALEIGH,NC,27604,POINT (-8746350.63865 4277818.43990)
